In [ ]:
import os
import pandas as pd
import geopandas as gpd
import plotly.express as px

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## See how geographies change over time
Many geographies change over time. For example, every 10 years, congressional district change during a process called redistricting. Understanding how these geographies are changing can help inform stories. 

Let's take a look at those congressional districts for Maryland. 

*NOTE: You may be asking yourself why I'm pulling data from 2022 and 2019 but calling it `md_cd_2020` and `md_cd_2010`, respectively. While these shapes are redraw every 10 years, they are also improved for geographic accuracy by the Census Bureau every few years. They boundaries from the 2010 file and the 2019 file will be the same boundaries, but the 2019 files will be more geographically precise.*

In [ ]:
md_cd_2020 = gpd.read_file('https://www2.census.gov/geo/tiger/TIGER2022/CD/tl_2022_24_cd118.zip')

#these older files only come on the us level so we'll need to filter once we download
us_cd_2010 = gpd.read_file('https://www2.census.gov/geo/tiger/TIGER2019/CD/tl_2019_us_cd116.zip')
md_cd_2010 = us_cd_2010.loc[us_cd_2010['STATEFP'] == '24']

In [ ]:
md_cd_2020.plot()

In [ ]:
md_cd_2010.plot()

I can SEE that they are different, but maybe I want to target the areas that have changed so i can see where they are exactly. Maybe I can even attach some data to these shapes and see who is being affected by the changes?

In [ ]:
#do the intersection
overlay = gpd.overlay(md_cd_2010, md_cd_2020, how='intersection')
overlay['changed'] = overlay.apply(lambda x: 'yes' if x['NAMELSAD'] != x['NAMELSAD20'] else 'no', axis=1)
overlay['id'] = overlay['NAMELSAD'] + ' - ' + overlay['NAMELSAD20']

#create a df that's just the data
overlay_df = overlay[['id','NAMELSAD','NAMELSAD20','changed']]

#Changing the projection so it works nicely with plotly
overlay = overlay.to_crs('EPSG:4326')

#Setting the index because plotly needs this
overlay.set_index('id',inplace=True)

#viz it
fig = px.choropleth_mapbox(overlay_df, geojson=overlay, 
                           locations='id',
                           color='changed',
                           color_discrete_map={'no':'#189196','yes':'#D1365E'},
                           hover_data=['NAMELSAD','NAMELSAD20'],
                           mapbox_style="carto-positron",
                           zoom=6, center = {"lat": 39.23274, "lon": -77.03450},
                           opacity=0.5,
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

## Getting data into custom shapes
We saw earlier that our Baltimore neighborhoods shapefile (provided by the city) already has some helpful data attached, but what if we want to add more data? Or maybe we want to add the same data but from sources we can verify the dates and quality of?

We can do this pretty easily with code! 

In [ ]:
#first we import the data. They dtype thing is just to make sure our join column is of the right type
tracts_shp = gpd.read_file('../GIS/tl_2022_47_tract.zip', dtype={'TRACTCE':str})
nash_redline = gpd.read_file('../GIS/redlining/TNNashville19XX.zip')


#our shapes need to be in the same projection for this
tracts_shp.to_crs('EPSG:2274',inplace=True)
nash_redline.to_crs('EPSG:2274',inplace=True)


#we'll want to join demographic data to our tracts shape now
demo_tract_df = pd.read_csv('../data/acs5-2021-demo-47-tracts.csv', dtype={'tract':str})
demo_tract_shp_df = tracts_shp.merge(demo_tract_df,left_on='TRACTCE',right_on='tract',how='left')


#and we'll need to calculate the area of the tracts before and after we do the intersection so we can
#calculate the actual estimated count of people of different races in each tract segmemt.
#we're multiplying by 3.587E-8 to turn unmanagably large feet into manageable miles!
demo_tract_shp_df['pre_area'] = demo_tract_shp_df['geometry'].area * 3.587E-8


#now let's do that intersection!
tracts_redlining = gpd.overlay(demo_tract_shp_df, nash_redline, how='intersection')


#and calculate the area post intersection.  
tracts_redlining['post_area'] = tracts_redlining['geometry'].area * 3.587E-8


#from the pre and post area calculations we create a percentage
tracts_redlining['pct_area'] = tracts_redlining['post_area']/demo_tract_shp_df['pre_area']

#and then we multiply our tract population counts by that percentage
#to get an estimated count of people who live in the segments of the tract
#that fall within each redlining district
tracts_redlining['post_pop'] = tracts_redlining['pop'] * tracts_redlining['pct_area']
tracts_redlining['post_pop1r'] = tracts_redlining['pop_1r'] * tracts_redlining['pct_area']
tracts_redlining['post_white'] = tracts_redlining['pop_1r_white'] * tracts_redlining['pct_area']
tracts_redlining['post_black'] = tracts_redlining['pop_1r_black'] * tracts_redlining['pct_area']
tracts_redlining['post_aian'] = tracts_redlining['pop_1r_aian'] * tracts_redlining['pct_area']
tracts_redlining['post_asian'] = tracts_redlining['pop_1r_asian'] * tracts_redlining['pct_area']
tracts_redlining['post_nhpi'] = tracts_redlining['pop_1r_nhpi'] * tracts_redlining['pct_area']

#and for display purposes, let's calculate one more value... post_pct_poc
tracts_redlining['post_pct_poc'] = (tracts_redlining['post_pop1r'] - tracts_redlining['post_white'])/tracts_redlining['post_pop1r']


#and output as a file so we can checkout in QGIS!
tracts_redlining.to_file('../GIS/analyzed/nashville-redlining-tracts-int.shp')

tracts_redlining.head()